# Loading data with DisjointTimeBasedCesnetDataset

### Import

In [1]:
from tqdm import tqdm
import logging

from cesnet_tszoo.utils.enums import AgreggationType, SourceType, TimeFormat, DatasetType
from cesnet_tszoo.datasets import CESNET_TimeSeries24
from cesnet_tszoo.configs import DisjointTimeBasedConfig # Disjoint dataset MUST use DisjointTimeBasedConfig

### Setting logger

In [2]:
logging.basicConfig(
    level=logging.INFO,
    format="[%(asctime)s][%(name)s][%(levelname)s] - %(message)s")

### Preparing dataset

In [3]:
disjoint_dataset = CESNET_TimeSeries24.get_dataset(data_root="/some_directory/", source_type=SourceType.INSTITUTION_SUBNETS, aggregation=AgreggationType.AGG_1_HOUR, dataset_type=DatasetType.DISJOINT_TIME_BASED, display_details=True)

[2025-11-14 18:34:01,788][cesnet_dataset][INFO] - Dataset is disjoint_time_based. Use cesnet_tszoo.configs.DisjointTimeBasedConfig



Dataset details:

    AgreggationType.AGG_1_HOUR
        Time indices: range(0, 6717)
        Datetime: (datetime.datetime(2023, 10, 9, 0, 0, tzinfo=datetime.timezone.utc), datetime.datetime(2024, 7, 14, 21, 0, tzinfo=datetime.timezone.utc))

    SourceType.INSTITUTION_SUBNETS
        Time series indices: [0 1 2 3 4 ... 543 544 545 546 547], Length=548; use 'get_available_ts_indices' for full list
        Features with default values: {'n_flows': 0, 'n_packets': 0, 'n_bytes': 0, 'tcp_udp_ratio_packets': 0.5, 'tcp_udp_ratio_bytes': 0.5, 'dir_ratio_packets': 0.5, 'dir_ratio_bytes': 0.5, 'avg_duration': 0, 'avg_ttl': 0, 'sum_n_dest_asn': 0, 'avg_n_dest_asn': 0, 'std_n_dest_asn': 0, 'sum_n_dest_ports': 0, 'avg_n_dest_ports': 0, 'std_n_dest_ports': 0, 'sum_n_dest_ip': 0, 'avg_n_dest_ip': 0, 'std_n_dest_ip': 0}
        
        Additional data: ['ids_relationship', 'weekends_and_holidays']
        


### Loading data with DataLoader

- Load data using Pytorch Dataloader.
- Last batch is never dropped (unless sliding_window is used).
- Workers affect how many processes will be used for loading data for specific set.
    - Workers set to 0, means loading will be ran on main process.
    - Set workers can be overriden in `get_*_dataloader` with parameter `workers`.
- Batch size affect how many times for every time series will be in one batch (differs when sliding window is used).
- Batch consists of: (only when sliding window is not used).
    - When `time_format` is not TimeFormat.DATETIME, then batch is one Numpy array of shape `(train_ts/val_ts/test_ts, batch_size, features_to_take + used ids)`.
    - When `time_format` is TimeFormat.DATETIME, then batch is a tuple: (Numpy array of shape `(train_ts/val_ts/test_ts, batch_size, features_to_take + used ids (without time))`, Numpy array of shape `(batch_size)`)

In [4]:
config = DisjointTimeBasedConfig(train_ts=0.5, val_ts=0.2, test_ts=0.1, train_time_period=0.5, val_time_period=0.3, test_time_period=0.2, features_to_take="all", time_format=TimeFormat.ID_TIME,
                         train_workers=0, val_workers=0, test_workers=0, init_workers=0, random_state=111,
                         train_batch_size=32, val_batch_size=64, test_batch_size=128)
disjoint_dataset.set_dataset_config_and_initialize(config, display_config_details="text", workers=0)

[2025-11-14 18:34:01,794][disjoint_time_based_config][INFO] - Quick validation succeeded.
[2025-11-14 18:34:01,810][cesnet_dataset][INFO] - Updating config for train set and fitting values.
[2025-11-14 18:34:01,810][cesnet_dataset][INFO] - Starting fitting cycle 1/1.
100%|██████████| 274/274 [00:00<00:00, 1436.76it/s]
[2025-11-14 18:34:02,023][cesnet_dataset][INFO] - Updating config for val set.
100%|██████████| 109/109 [00:00<00:00, 1369.27it/s]
[2025-11-14 18:34:02,115][cesnet_dataset][INFO] - Updating config for test set.
100%|██████████| 54/54 [00:00<00:00, 1315.03it/s]
[2025-11-14 18:34:02,163][cesnet_dataset][INFO] - Dataset initialization complete. Configuration updated.
[2025-11-14 18:34:02,163][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Train time series IDs: [253 277 132 114  76 ...   7 118 322 275  86], Length=274
        Val time series IDs: [434  61  38  53 522 ... 540  52 111 370 309], Length=109
        Test time series IDs: [339 133 457  97  18 ... 359 505 328 192 181], Length=54
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: range(3359, 5374)
        Test time periods: range(5374, 6717)
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 

You can also change set batch sizes later with `update_dataset_config_and_initialize` or `set_batch_sizes`.

In [5]:
disjoint_dataset.update_dataset_config_and_initialize(train_batch_size=33, val_batch_size=65, test_batch_size="config")
# Or
disjoint_dataset.set_batch_sizes(train_batch_size=33, val_batch_size=65, test_batch_size="config")

[2025-11-14 18:34:02,172][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-11-14 18:34:02,175][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-11-14 18:34:02,175][cesnet_dataset][INFO] - Batch sizes has been changed successfuly.


You can also change set workers later with `update_dataset_config_and_initialize` or `set_workers`.

In [6]:
disjoint_dataset.update_dataset_config_and_initialize(train_workers=0, val_workers=0, test_workers=0, init_workers=0)
# Or
disjoint_dataset.set_workers(train_workers=0, val_workers=0, test_workers=0, init_workers=0)

[2025-11-14 18:34:02,182][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-11-14 18:34:02,184][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-11-14 18:34:02,185][cesnet_dataset][INFO] - Workers has been changed successfuly.


#### Train set

- Affected by `train_batch_size`.
- Affected by `train_workers`.

In [7]:
dataloader = disjoint_dataset.get_train_dataloader(workers="config")

batches = []

for batch in tqdm(dataloader):
    batches.append(batch)
    
display(batches[0].shape)

[2025-11-14 18:34:02,198][cesnet_dataset][INFO] - Created new cached train_dataloader.
100%|██████████| 102/102 [00:02<00:00, 44.62it/s]


(274, 33, 20)

#### Val set

- Affected by `val_batch_size`.
- Affected by `val_workers`.

In [8]:
dataloader = disjoint_dataset.get_val_dataloader(workers="config")

batches = []

for batch in tqdm(dataloader):
    batches.append(batch)
    
display(batches[0].shape)

[2025-11-14 18:34:04,497][cesnet_dataset][INFO] - Created new cached val_dataloader.
100%|██████████| 31/31 [00:00<00:00, 65.54it/s]


(109, 65, 20)

#### Test set

- Affected by `test_batch_size`.
- Affected by `test_workers`.

In [9]:
dataloader = disjoint_dataset.get_test_dataloader(workers="config")

batches = []

for batch in tqdm(dataloader):
    batches.append(batch)
    
display(batches[0].shape)

[2025-11-14 18:34:04,986][cesnet_dataset][INFO] - Created new cached test_dataloader.
100%|██████████| 11/11 [00:00<00:00, 102.75it/s]


(54, 128, 20)

#### Using time_format=TimeFormat.DATETIME

In [10]:
config = DisjointTimeBasedConfig(train_ts=0.5, val_ts=0.2, test_ts=0.1, train_time_period=0.5, val_time_period=0.3, test_time_period=0.2, features_to_take="all", time_format=TimeFormat.DATETIME,
                         train_workers=0, val_workers=0, test_workers=0, init_workers=0, random_state=111,
                         train_batch_size=32, val_batch_size=64, test_batch_size=128)
disjoint_dataset.set_dataset_config_and_initialize(config, display_config_details="text", workers=0)

[2025-11-14 18:34:05,102][disjoint_time_based_config][INFO] - Quick validation succeeded.
[2025-11-14 18:34:05,121][cesnet_dataset][INFO] - Updating config for train set and fitting values.
[2025-11-14 18:34:05,121][cesnet_dataset][INFO] - Starting fitting cycle 1/1.
100%|██████████| 274/274 [00:00<00:00, 3007.86it/s]
[2025-11-14 18:34:05,228][cesnet_dataset][INFO] - Updating config for val set.
100%|██████████| 109/109 [00:00<00:00, 3673.83it/s]
[2025-11-14 18:34:05,264][cesnet_dataset][INFO] - Updating config for test set.
100%|██████████| 54/54 [00:00<00:00, 4143.74it/s]
[2025-11-14 18:34:05,279][cesnet_dataset][INFO] - Dataset initialization complete. Configuration updated.
[2025-11-14 18:34:05,280][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Train time series IDs: [253 277 132 114  76 ...   7 118 322 275  86], Length=274
        Val time series IDs: [434  61  38  53 522 ... 540  52 111 370 309], Length=109
        Test time series IDs: [339 133 457  97  18 ... 359 505 328 192 181], Length=54
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: range(3359, 5374)
        Test time periods: range(5374, 6717)
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 

In [11]:
dataloader = disjoint_dataset.get_train_dataloader(workers="config")

batches = []

for batch in tqdm(dataloader):
    batches.append(batch)
    
display(batches[0][0].shape) # data without time
display(batches[0][1].shape) # time

[2025-11-14 18:34:05,310][cesnet_dataset][INFO] - Created new cached train_dataloader.
100%|██████████| 105/105 [00:02<00:00, 46.50it/s]


(274, 32, 19)

(32,)

#### Specifying which time series to load

- Every `get_*_dataloader` has parameter `ts_id`.
    - When `ts_id` is None, then it returns as previous examples.
    - When `ts_id` is not None, then it returns only one time series of that specified id.

In [12]:
config = DisjointTimeBasedConfig(train_ts=[177, 176, 319, 267], val_ts=None, test_ts=None, train_time_period=0.5, features_to_take="all", time_format=TimeFormat.ID_TIME,
                         train_workers=0, val_workers=0, test_workers=0, init_workers=0, random_state=111,
                         train_batch_size=32, val_batch_size=64, test_batch_size=128)
disjoint_dataset.set_dataset_config_and_initialize(config, display_config_details="text", workers=0)

[2025-11-14 18:34:07,577][disjoint_time_based_config][INFO] - Quick validation succeeded.
[2025-11-14 18:34:07,583][cesnet_dataset][INFO] - Updating config for train set and fitting values.
[2025-11-14 18:34:07,584][cesnet_dataset][INFO] - Starting fitting cycle 1/1.
100%|██████████| 4/4 [00:00<00:00, 1140.38it/s]
[2025-11-14 18:34:07,590][cesnet_dataset][INFO] - Dataset initialization complete. Configuration updated.
[2025-11-14 18:34:07,591][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Train time series IDs: [177 176 319 267], Length=4
        Val time series IDs: None
        Test time series IDs: None
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: None
        Test time periods: None
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.5 0.5 0.5 0.  0. ]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    Sliding window
      

In [13]:
dataloader = disjoint_dataset.get_train_dataloader(ts_id=177 , workers="config",)

batches = []

for batch in tqdm(dataloader):
    batches.append(batch)
    
display(batches[0].shape)

[2025-11-14 18:34:07,599][cesnet_dataset][INFO] - Created new cached train_dataloader.
100%|██████████| 105/105 [00:00<00:00, 2048.74it/s]


(1, 32, 20)

#### Sliding window

- When `sliding_window_prediction_size` is set then `sliding_window_size` must be set too if you want to use sliding window.
- Batch sizes are used for background caching.
- Batch consists of:
    - When `time_format` is not TimeFormat.DATETIME, then batch is a tuple: <br>
    ( <br>
        Numpy array of shape `(train_ts/val_ts/test_ts, sliding_window_size, features_to_take + used ids)`, <br>
        Numpy array of shape `(train_ts/val_ts/test_ts, sliding_window_prediction_size, features_to_take + used ids)` <br>
    ).
    - When `time_format` is TimeFormat.DATETIME, then batch is a tuple: <br>
    ( <br>
        Numpy array of shape `(train_ts/val_ts/test_ts, sliding_window_size, features_to_take + used ids (without time))`, <br>
        Numpy array of shape `(train_ts/val_ts/test_ts, sliding_window_prediction_size, features_to_take + used ids (without time))`, <br>
        Numpy array of shape `(sliding_window_size)` of time, <br>
        Numpy array of shape `(sliding_window_prediction_size)` of time, <br>
    ).
- You can modify sliding window step size with `sliding_window_step`
- You can use `set_shared_size` to set how many times time periods should share.
    - `val_time_period` takes from `train_time_period`
    - `test_time_period` takes from `val_time_period` or `train_time_period`

In [14]:
config = DisjointTimeBasedConfig(train_ts=0.5, val_ts=0.2, test_ts=0.1, train_time_period=range(0, 1000), val_time_period=range(1000, 1500), test_time_period=range(1500, 2000), features_to_take=["n_flows"], time_format=TimeFormat.ID_TIME,
                         train_workers=0, val_workers=0, test_workers=0, init_workers=0, random_state=111,
                         train_batch_size=32, val_batch_size=64, test_batch_size=128,
                         sliding_window_size=22, sliding_window_prediction_size=2, sliding_window_step=2, set_shared_size=0.05)
disjoint_dataset.set_dataset_config_and_initialize(config, display_config_details="text", workers=0)

[2025-11-14 18:34:07,663][disjoint_time_based_config][INFO] - Quick validation succeeded.
[2025-11-14 18:34:07,677][cesnet_dataset][INFO] - Updating config for train set and fitting values.
[2025-11-14 18:34:07,677][cesnet_dataset][INFO] - Starting fitting cycle 1/1.
100%|██████████| 274/274 [00:00<00:00, 8975.49it/s]
[2025-11-14 18:34:07,716][cesnet_dataset][INFO] - Updating config for val set.
100%|██████████| 109/109 [00:00<00:00, 7260.96it/s]
[2025-11-14 18:34:07,736][cesnet_dataset][INFO] - Updating config for test set.
100%|██████████| 54/54 [00:00<00:00, 7190.92it/s]
[2025-11-14 18:34:07,745][cesnet_dataset][INFO] - Dataset initialization complete. Configuration updated.
[2025-11-14 18:34:07,745][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Train time series IDs: [253 277 132 114  76 ...   7 118 322 275  86], Length=274
        Val time series IDs: [434  61  38  53 522 ... 540  52 111 370 309], Length=109
        Test time series IDs: [339 133 457  97  18 ... 359 505 328 192 181], Length=54
    Time periods
        Train time periods: range(0, 1000)
        Val time periods: range(665, 1500)
        Test time periods: range(1165, 2000)
    Features
        Taken features: ['n_flows']
        Default values: [0.]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    Sliding window
        Sliding window size: 22
        Sliding window prediction size: 2
        Sliding window step size: 2
    Fillers
        Filler type: NoFiller
    Transformers
        Transformer type: NoTransformer
    Anomal

In [15]:
dataloader = disjoint_dataset.get_train_dataloader(workers="config")

batches = []

for sliding_window, sliding_window_prediction in tqdm(dataloader):
    batches.append((sliding_window, sliding_window_prediction))

[2025-11-14 18:34:07,761][cesnet_dataset][INFO] - Created new cached train_dataloader.
100%|██████████| 489/489 [00:00<00:00, 1154.15it/s]


You can also change sliding window parameters later with `update_dataset_config_and_initialize` or `set_sliding_window`.

In [16]:
disjoint_dataset.update_dataset_config_and_initialize(sliding_window_size=22, sliding_window_prediction_size=3, sliding_window_step="config", set_shared_size="config", workers=0)
# Or
disjoint_dataset.set_sliding_window(sliding_window_size=22, sliding_window_prediction_size=3, sliding_window_step="config", set_shared_size="config", workers=0)

[2025-11-14 18:34:08,193][cesnet_dataset][INFO] - Destroyed cached train_dataloader.
[2025-11-14 18:34:08,193][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-11-14 18:34:08,196][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-11-14 18:34:08,197][cesnet_dataset][INFO] - Sliding window values has been changed successfuly.


##### Using time_format=TimeFormat.DATETIME

In [17]:
config = DisjointTimeBasedConfig(train_ts=0.5, val_ts=0.2, test_ts=0.1, train_time_period=range(0, 1000), val_time_period=range(978, 1500), test_time_period=range(1478, 2000), features_to_take=["n_flows"], time_format=TimeFormat.DATETIME,
                         train_workers=0, val_workers=0, test_workers=0, init_workers=0, random_state=111,
                         train_batch_size=32, val_batch_size=64, test_batch_size=128,
                         sliding_window_size=22, sliding_window_prediction_size=2, sliding_window_step=2, set_shared_size=100)
disjoint_dataset.set_dataset_config_and_initialize(config, display_config_details="text", workers=0)

[2025-11-14 18:34:08,203][disjoint_time_based_config][INFO] - Quick validation succeeded.
[2025-11-14 18:34:08,218][cesnet_dataset][INFO] - Updating config for train set and fitting values.
[2025-11-14 18:34:08,218][cesnet_dataset][INFO] - Starting fitting cycle 1/1.
100%|██████████| 274/274 [00:00<00:00, 10332.75it/s]
[2025-11-14 18:34:08,256][cesnet_dataset][INFO] - Updating config for val set.
100%|██████████| 109/109 [00:00<00:00, 8052.90it/s]
[2025-11-14 18:34:08,274][cesnet_dataset][INFO] - Updating config for test set.
100%|██████████| 54/54 [00:00<00:00, 7184.53it/s]
[2025-11-14 18:34:08,283][cesnet_dataset][INFO] - Dataset initialization complete. Configuration updated.
[2025-11-14 18:34:08,284][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Train time series IDs: [253 277 132 114  76 ...   7 118 322 275  86], Length=274
        Val time series IDs: [434  61  38  53 522 ... 540  52 111 370 309], Length=109
        Test time series IDs: [339 133 457  97  18 ... 359 505 328 192 181], Length=54
    Time periods
        Train time periods: range(0, 1000)
        Val time periods: range(900, 1500)
        Test time periods: range(1400, 2000)
    Features
        Taken features: ['n_flows']
        Default values: [0.]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.DATETIME
    Sliding window
        Sliding window size: 22
        Sliding window prediction size: 2
        Sliding window step size: 2
    Fillers
        Filler type: NoFiller
    Transformers
        Transformer type: NoTransformer
    Anoma

In [18]:
dataloader = disjoint_dataset.get_train_dataloader(workers="config")

batches = []

for sliding_window, sliding_window_prediction, sliding_window_times, sliding_window_prediction_times in tqdm(dataloader):
    batches.append((sliding_window, sliding_window_prediction, sliding_window_times, sliding_window_prediction_times))

[2025-11-14 18:34:08,304][cesnet_dataset][INFO] - Created new cached train_dataloader.
100%|██████████| 489/489 [00:00<00:00, 1208.54it/s]


### Loading data as Dataframe

- Batch size has no effect.
- Sliding window has no effect.
- Returns every time series in `train_ts/val_ts/test_ts` with sets specified time period.
- Data is returned as Pandas Dataframe.
- Workers affect how many processes will be used for loading data for specific set.
    - Workers set to 0, means loading will be ran on main process.
    - Set workers can be overriden in `get_*_df` with parameter `workers`.

In [19]:
config = DisjointTimeBasedConfig(train_ts=0.5, val_ts=0.2, test_ts=0.1, train_time_period=0.5, val_time_period=0.3, test_time_period=0.2, features_to_take="all", time_format=TimeFormat.ID_TIME,
                         train_workers=0, val_workers=0, test_workers=0, init_workers=0, random_state=111)
disjoint_dataset.set_dataset_config_and_initialize(config, display_config_details="text", workers=0)

[2025-11-14 18:34:08,715][disjoint_time_based_config][INFO] - Quick validation succeeded.
[2025-11-14 18:34:08,731][cesnet_dataset][INFO] - Updating config for train set and fitting values.
[2025-11-14 18:34:08,731][cesnet_dataset][INFO] - Starting fitting cycle 1/1.
100%|██████████| 274/274 [00:00<00:00, 2923.03it/s]
[2025-11-14 18:34:08,834][cesnet_dataset][INFO] - Updating config for val set.
100%|██████████| 109/109 [00:00<00:00, 3351.70it/s]
[2025-11-14 18:34:08,872][cesnet_dataset][INFO] - Updating config for test set.
100%|██████████| 54/54 [00:00<00:00, 3721.23it/s]
[2025-11-14 18:34:08,889][cesnet_dataset][INFO] - Dataset initialization complete. Configuration updated.
[2025-11-14 18:34:08,889][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Train time series IDs: [253 277 132 114  76 ...   7 118 322 275  86], Length=274
        Val time series IDs: [434  61  38  53 522 ... 540  52 111 370 309], Length=109
        Test time series IDs: [339 133 457  97  18 ... 359 505 328 192 181], Length=54
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: range(3359, 5374)
        Test time periods: range(5374, 6717)
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 

#### Train set

- Affected by `train_workers`.

In [20]:
df = disjoint_dataset.get_train_df(as_single_dataframe=True, workers="config")
dfs = disjoint_dataset.get_train_df(as_single_dataframe=False, workers="config")

df.head(10)

,id_institution_subnet,id_time,n_flows,n_packets,n_bytes,sum_n_dest_asn,avg_n_dest_asn,std_n_dest_asn,sum_n_dest_ports,avg_n_dest_ports,std_n_dest_ports,sum_n_dest_ip,avg_n_dest_ip,std_n_dest_ip,tcp_udp_ratio_packets,tcp_udp_ratio_bytes,dir_ratio_packets,dir_ratio_bytes,avg_duration,avg_ttl
0,253.0,0.0,3993.0,12262.0,4.819489e+06,3335.0,4.87,2.14,3662.0,5.35,2.49,3705.0,5.41,2.640000,0.950195,0.930176,0.409912,0.409912,0.83,141.399994
1,253.0,1.0,4333.0,10780.0,3.196162e+06,3564.0,5.09,2.48,4008.0,5.73,3.14,3996.0,5.71,3.070000,0.930176,0.910156,0.399902,0.389893,0.75,136.699997
2,253.0,2.0,5194.0,76853.0,6.782261e+07,3361.0,4.92,2.55,3647.0,5.34,2.66,3867.0,5.66,4.400000,0.930176,0.910156,0.419922,0.409912,0.88,134.429993
3,253.0,3.0,6587.0,649647.0,7.588314e+08,3066.0,4.65,4.29,3088.0,4.68,2.82,4403.0,6.67,21.120001,0.939941,0.919922,0.459961,0.439941,1.28,129.429993
4,253.0,4.0,8369.0,544770.0,4.646289e+08,2565.0,4.04,5.28,2397.0,3.77,2.37,4747.0,7.48,31.049999,0.939941,0.919922,0.479980,0.469971,0.75,125.849998
5,253.0,5.0,9365.0,1232213.0,1.244550e+09,2460.0,4.05,6.51,2143.0,3.53,2.38,5139.0,8.47,36.139999,0.950195,0.930176,0.489990,0.479980,1.21,128.309998
6,253.0,6.0,9451.0,1247653.0,1.334478e+09,2496.0,4.19,6.41,2221.0,3.73,2.40,5186.0,8.70,39.020000,0.939941,0.919922,0.469971,0.459961,1.04,131.380005
7,253.0,7.0,8566.0,743959.0,7.116519e+08,2578.0,4.17,5.88,2440.0,3.95,2.42,4899.0,7.93,33.709999,0.930176,0.919922,0.459961,0.449951,1.52,130.350006
8,253.0,8.0,8607.0,947064.0,9.099148e+08,2568.0,4.14,6.18,2267.0,3.65,2.18,5066.0,8.16,35.410000,0.930176,0.899902,0.479980,0.459961,0.94,123.669998
9,253.0,9.0,7792.0,425931.0,3.531234e+08,2469.0,4.07,5.25,2290.0,3.78,2.48,4718.0,7.79,30.219999,0.939941,0.919922,0.469971,0.459961,1.04,129.100006


In [21]:
len(dfs) # every time series has its own dataframe

274

#### Val set

- Affected by `val_workers`.

In [22]:
df = disjoint_dataset.get_val_df(as_single_dataframe=True, workers="config")
dfs = disjoint_dataset.get_val_df(as_single_dataframe=False, workers="config")

df.head(10)

,id_institution_subnet,id_time,n_flows,n_packets,n_bytes,sum_n_dest_asn,avg_n_dest_asn,std_n_dest_asn,sum_n_dest_ports,avg_n_dest_ports,std_n_dest_ports,sum_n_dest_ip,avg_n_dest_ip,std_n_dest_ip,tcp_udp_ratio_packets,tcp_udp_ratio_bytes,dir_ratio_packets,dir_ratio_bytes,avg_duration,avg_ttl
0,434.0,3359.0,1662.0,14788.0,1653828.0,191.0,9.10,7.520000,299.0,14.24,15.21,289.0,13.760000,12.500000,0.750000,0.740234,0.489990,0.500000,42.430000,160.210007
1,434.0,3360.0,1486.0,16678.0,2103699.0,194.0,8.08,6.330000,301.0,12.54,12.80,300.0,12.500000,9.990000,0.750000,0.700195,0.520020,0.529785,43.410000,155.820007
2,434.0,3361.0,1347.0,46514.0,28687741.0,190.0,8.64,7.080000,303.0,13.77,15.57,289.0,13.140000,11.370000,0.759766,0.740234,0.459961,0.449951,36.599998,156.089996
3,434.0,3362.0,1676.0,14786.0,1905753.0,186.0,8.09,6.810000,285.0,12.39,13.34,288.0,12.520000,11.460000,0.759766,0.740234,0.489990,0.500000,30.870001,153.330002
4,434.0,3363.0,1516.0,106105.0,118491246.0,213.0,9.26,8.200000,281.0,12.22,12.89,342.0,14.870000,17.250000,0.779785,0.770020,0.419922,0.409912,41.570000,154.720001
5,434.0,3364.0,1098.0,60222.0,26692817.0,186.0,9.79,9.690000,203.0,10.68,11.03,316.0,16.629999,18.080000,0.740234,0.720215,0.589844,0.509766,46.820000,167.020004
6,434.0,3365.0,1297.0,75205.0,52164277.0,216.0,10.29,11.160000,178.0,8.48,8.31,576.0,27.430000,40.029999,0.740234,0.729980,0.479980,0.389893,44.680000,155.020004
7,434.0,3366.0,1879.0,129166.0,91007593.0,272.0,13.60,17.120001,168.0,8.40,8.98,731.0,36.549999,53.270000,0.660156,0.620117,0.500000,0.399902,47.880001,156.360001
8,434.0,3367.0,2432.0,261921.0,198660129.0,308.0,14.00,18.750000,179.0,8.14,9.78,1053.0,47.860001,74.930000,0.750000,0.729980,0.600098,0.500000,35.660000,171.500000
9,434.0,3368.0,2080.0,130488.0,107103503.0,309.0,15.45,18.200001,175.0,8.75,8.12,1038.0,51.900002,73.879997,0.729980,0.669922,0.509766,0.419922,59.970001,162.960007


In [23]:
len(dfs) # every time series has its own dataframe

109

#### Test set

- Affected by `test_workers`.

In [24]:
df = disjoint_dataset.get_test_df(as_single_dataframe=True, workers="config")
dfs = disjoint_dataset.get_test_df(as_single_dataframe=False, workers="config")

df.head(10)

,id_institution_subnet,id_time,n_flows,n_packets,n_bytes,sum_n_dest_asn,avg_n_dest_asn,std_n_dest_asn,sum_n_dest_ports,avg_n_dest_ports,std_n_dest_ports,sum_n_dest_ip,avg_n_dest_ip,std_n_dest_ip,tcp_udp_ratio_packets,tcp_udp_ratio_bytes,dir_ratio_packets,dir_ratio_bytes,avg_duration,avg_ttl
0,339.0,5374.0,65.0,472.0,137858.0,26.0,2.89,1.96,29.0,3.22,2.49,58.0,6.44,4.10,0.340088,0.399902,0.429932,0.489990,38.669998,150.850006
1,339.0,5375.0,83.0,298.0,82157.0,35.0,2.92,1.08,33.0,2.75,1.22,79.0,6.58,3.65,0.459961,0.500000,0.449951,0.479980,18.680000,133.630005
2,339.0,5376.0,105.0,410.0,114437.0,34.0,2.83,1.53,35.0,2.92,1.51,99.0,8.25,4.58,0.370117,0.409912,0.439941,0.489990,15.780000,146.789993
3,339.0,5377.0,114.0,319.0,52219.0,34.0,3.40,1.07,38.0,3.80,1.40,105.0,10.50,5.17,0.330078,0.360107,0.399902,0.449951,19.360001,148.139999
4,339.0,5378.0,98.0,258.0,34198.0,28.0,2.55,1.29,21.0,1.91,1.30,93.0,8.45,3.80,0.180054,0.219971,0.449951,0.419922,14.790000,158.919998
5,339.0,5379.0,131.0,469.0,89807.0,39.0,3.25,2.05,43.0,3.58,2.54,120.0,10.00,5.46,0.270020,0.350098,0.449951,0.439941,13.960000,146.009995
6,339.0,5380.0,53.0,230.0,69059.0,21.0,2.62,1.30,23.0,2.88,1.55,49.0,6.12,5.11,0.520020,0.580078,0.479980,0.549805,21.330000,113.879997
7,339.0,5381.0,18.0,143.0,33279.0,12.0,2.40,1.14,12.0,2.40,1.14,16.0,3.20,1.79,0.520020,0.520020,0.500000,0.489990,70.900002,81.410004
8,339.0,5382.0,40.0,234.0,57090.0,23.0,2.88,1.13,24.0,3.00,0.93,39.0,4.88,1.96,0.459961,0.489990,0.360107,0.419922,20.629999,173.839996
9,339.0,5383.0,73.0,243.0,49791.0,33.0,3.30,1.06,26.0,2.60,0.97,70.0,7.00,3.50,0.310059,0.320068,0.340088,0.340088,16.959999,155.710007


In [25]:
len(dfs) # every time series has its own dataframe

54

### Loading data as singular Numpy array 

- Batch size has no effect.
- Sliding window has no effect.
- Returns every time series in `train_ts/val_ts/test_ts` with sets specified time period.
- Data is returned as one Numpy array.
- Follows similar rules to Dataloader batches, regarding shape (excluding sliding window parameters).
- Workers affect how many processes will be used for loading data for specific set.
    - Workers set to 0, means loading will be ran on main process.
    - Set workers can be overriden in `get_*_numpy` with parameter `workers`.

In [26]:
config = DisjointTimeBasedConfig(train_ts=0.5, val_ts=0.2, test_ts=0.1, train_time_period=0.5, val_time_period=0.3, test_time_period=0.2, features_to_take="all", time_format=TimeFormat.ID_TIME,
                         train_workers=0, val_workers=0, test_workers=0, init_workers=0, random_state=111)
disjoint_dataset.set_dataset_config_and_initialize(config, display_config_details="text", workers=0)

[2025-11-14 18:34:09,727][disjoint_time_based_config][INFO] - Quick validation succeeded.
[2025-11-14 18:34:09,743][cesnet_dataset][INFO] - Updating config for train set and fitting values.
[2025-11-14 18:34:09,743][cesnet_dataset][INFO] - Starting fitting cycle 1/1.
100%|██████████| 274/274 [00:00<00:00, 2820.25it/s]
[2025-11-14 18:34:09,849][cesnet_dataset][INFO] - Updating config for val set.
100%|██████████| 109/109 [00:00<00:00, 3756.48it/s]
[2025-11-14 18:34:09,884][cesnet_dataset][INFO] - Updating config for test set.
100%|██████████| 54/54 [00:00<00:00, 3999.02it/s]
[2025-11-14 18:34:09,899][cesnet_dataset][INFO] - Dataset initialization complete. Configuration updated.
[2025-11-14 18:34:09,899][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Train time series IDs: [253 277 132 114  76 ...   7 118 322 275  86], Length=274
        Val time series IDs: [434  61  38  53 522 ... 540  52 111 370 309], Length=109
        Test time series IDs: [339 133 457  97  18 ... 359 505 328 192 181], Length=54
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: range(3359, 5374)
        Test time periods: range(5374, 6717)
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 

#### Train set

- Affected by `train_workers`.

In [27]:
numpy_array = disjoint_dataset.get_train_numpy(workers="config")

display(numpy_array.shape)

(274, 3359, 20)

#### Val set

- Affected by `val_workers`.

In [28]:
numpy_array = disjoint_dataset.get_val_numpy(workers="config")

display(numpy_array.shape)

(109, 2015, 20)

#### Test set

- Affected by `test_workers`.

In [29]:
numpy_array = disjoint_dataset.get_test_numpy(workers="config")

display(numpy_array.shape)

(54, 1343, 20)

#### Using time_format=TimeFormat.DATETIME

In [30]:
config = DisjointTimeBasedConfig(train_ts=0.5, val_ts=0.2, test_ts=0.1, train_time_period=0.5, val_time_period=0.3, test_time_period=0.2, features_to_take="all", time_format=TimeFormat.DATETIME,
                         train_workers=0, val_workers=0, test_workers=0, init_workers=0, random_state=111)
disjoint_dataset.set_dataset_config_and_initialize(config, display_config_details="text", workers=0)

[2025-11-14 18:34:10,365][disjoint_time_based_config][INFO] - Quick validation succeeded.
[2025-11-14 18:34:10,384][cesnet_dataset][INFO] - Updating config for train set and fitting values.
[2025-11-14 18:34:10,385][cesnet_dataset][INFO] - Starting fitting cycle 1/1.
100%|██████████| 274/274 [00:00<00:00, 2913.06it/s]
[2025-11-14 18:34:10,495][cesnet_dataset][INFO] - Updating config for val set.
100%|██████████| 109/109 [00:00<00:00, 3605.49it/s]
[2025-11-14 18:34:10,531][cesnet_dataset][INFO] - Updating config for test set.
100%|██████████| 54/54 [00:00<00:00, 3998.95it/s]
[2025-11-14 18:34:10,547][cesnet_dataset][INFO] - Dataset initialization complete. Configuration updated.
[2025-11-14 18:34:10,547][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Train time series IDs: [253 277 132 114  76 ...   7 118 322 275  86], Length=274
        Val time series IDs: [434  61  38  53 522 ... 540  52 111 370 309], Length=109
        Test time series IDs: [339 133 457  97  18 ... 359 505 328 192 181], Length=54
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: range(3359, 5374)
        Test time periods: range(5374, 6717)
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 

In [31]:
numpy_array, times = disjoint_dataset.get_train_numpy(workers="config")

display(numpy_array.shape)
display(times)

(274, 3359, 19)

array([datetime.datetime(2023, 10, 9, 0, 0, tzinfo=datetime.timezone.utc),
       datetime.datetime(2023, 10, 9, 1, 0, tzinfo=datetime.timezone.utc),
       datetime.datetime(2023, 10, 9, 2, 0, tzinfo=datetime.timezone.utc),
       ...,
       datetime.datetime(2024, 2, 25, 20, 0, tzinfo=datetime.timezone.utc),
       datetime.datetime(2024, 2, 25, 21, 0, tzinfo=datetime.timezone.utc),
       datetime.datetime(2024, 2, 25, 22, 0, tzinfo=datetime.timezone.utc)],
      shape=(3359,), dtype=object)